# Experimentos para seleção de Modelo de regrassão

In [2]:
import sys
def category_encoders():
    package = 'category_encoders'
    try:
        return __import__(package)
    except ImportError:
        import sys
        !{sys.executable} -m pip install category_encoders
        return None

In [3]:
category_encoders()

<module 'category_encoders' from '/home/caio/anaconda3/lib/python3.8/site-packages/category_encoders/__init__.py'>

In [4]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
import category_encoders as ce
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

In [5]:
df = pd.read_csv("../raw_data.csv")
# df = pd.read_csv("../final_dataset.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3917 entries, 0 to 3916
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Editora       3917 non-null   object 
 1   ISBN          3917 non-null   object 
 2   Ano           3917 non-null   float64
 3   Edição        3917 non-null   float64
 4   Origem        3917 non-null   object 
 5   Encadernação  3917 non-null   object 
 6   Idioma        3917 non-null   object 
 7   País          3917 non-null   object 
 8   Páginas       3917 non-null   float64
 9   preço         3917 non-null   float64
 10  nome          3917 non-null   object 
 11  categoria     3917 non-null   object 
 12  Autores       3916 non-null   object 
dtypes: float64(4), object(9)
memory usage: 397.9+ KB


## Testando modelo Base line com apenas features numéricas

In [6]:
X = df[['Ano','Edição','Páginas']]
y = df['preço']

In [7]:
X['Ano'] = X['Ano'].astype(int)
X['Edição'] = X['Edição'].astype(int)
X['Páginas'] = X['Páginas'].astype(int)
X

<ipython-input-7-5ff5fe48f8ca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Ano'] = X['Ano'].astype(int)
<ipython-input-7-5ff5fe48f8ca>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Edição'] = X['Edição'].astype(int)
<ipython-input-7-5ff5fe48f8ca>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

,Ano,Edição,Páginas
0,2018,1,384
1,2014,1,384
2,2006,1,728
3,2021,1,288
4,2015,1,20
...,...,...,...
3912,2009,1,440
3913,2014,1,408
3914,2021,1,246
3915,2017,1,252


In [8]:
model = LinearRegression()
scores = cross_val_score(model, X, y, cv=5,scoring='r2')
print(scores)
print(f'Pontuação r2 média: {scores.mean()}')

[ 0.32465135  0.25719399  0.37555561 -0.00997366  0.31084054]
Pontuação r2 média: 0.2516535659354962


In [9]:
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
model = LinearRegression()
scores = cross_val_score(model, X_scaled, y, cv=5,scoring='r2')
print(scores)
print(f'Pontuação r2 média com normalização: {scores.mean()}')

[ 0.32465135  0.25719399  0.37555561 -0.00997366  0.31084054]
Pontuação r2 média com normalização: 0.25165356593549637


## Tratando features categóricas

### Codificando origem

In [10]:
encoder=ce.OneHotEncoder(cols='Origem',handle_unknown='return_nan',return_df=True,use_cat_names=True)
data_encoded = encoder.fit_transform(df)
data_encoded

/home/caio/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,Editora,ISBN,Ano,Edição,Origem_NACIONAL,Origem_IMPORTADO,Encadernação,Idioma,País,Páginas,preço,nome,categoria,Autores
0,GALERA RECORD,8501112526,2018.0,1.0,1.0,0.0,BROCHURA,PORTUGUÊS,BRASIL,384.0,49.90,UM DE NÓS ESTÁ MENTINDO,TEEN,KAREN MCMANUS
1,GLOBO LIVROS,8525057517,2014.0,1.0,1.0,0.0,BROCHURA,PORTUGUÊS,BRASIL,384.0,59.90,1808,HISTÓRIA,LAURENTINO GOMES
2,GLOBAL EDITORA,8526008692,2006.0,1.0,1.0,0.0,BROCHURA,PORTUGUÊS,BRASIL,728.0,130.00,CASA-GRANDE & SENZALA,HISTÓRIA,GILBERTO FREYRE
3,PLANETA DO BRASIL,6555352795,2021.0,1.0,1.0,0.0,BROCHURA,PORTUGUÊS,BRASIL,288.0,38.42,SOMBRA E OSSOS,INFANTIL,LEIGH BARDUGO
4,USBORNE,140959386X,2015.0,1.0,1.0,0.0,CAPA DURA,PORTUGUÊS,BRASIL,20.0,49.90,MINHAS PRIMEIRAS PALAVRAS,INFANTIL,FELICITY BROOKS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3912,ALTA BOOKS,8576082675,2009.0,1.0,1.0,0.0,BROCHURA,PORTUGUÊS,BRASIL,440.0,114.00,CÓDIGO LIMPO,INFORMÁTICA E TECNOLOGIA,ROBERT C. MARTIN
3913,ALTA BOOKS,8576088487,2014.0,1.0,1.0,0.0,BROCHURA,PORTUGUÊS,BRASIL,408.0,98.00,DESENVOLVIMENTO DE APLICATIVOS ANDROID PARA LE...,INFORMÁTICA E TECNOLOGIA,DONN FELKER; MICHAEL BURTON
3914,ALTA BOOKS,855081587X,2021.0,1.0,1.0,0.0,BROCHURA,PORTUGUÊS,BRASIL,246.0,49.90,PYTHON PARA DATA SCIENCE PARA LEIGOS,INFORMÁTICA E TECNOLOGIA,JOHN PAUL MUELLER; LUCA MASSARON
3915,MYTHOS EDITORA,8578672518,2017.0,1.0,1.0,0.0,CAPA DURA,PORTUGUÊS,BRASIL,252.0,84.90,ZENITH - VOLUME 02,HQS,GRANT MORRISON


### Codificando País

In [11]:
encoder=ce.OneHotEncoder(cols='País',handle_unknown='return_nan',return_df=True,use_cat_names=True)
data_encoded = encoder.fit_transform(data_encoded)
data_encoded

/home/caio/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,Editora,ISBN,Ano,Edição,Origem_NACIONAL,Origem_IMPORTADO,Encadernação,Idioma,País_BRASIL,País_ESTADOS UNIDOS,...,País_PORTUGAL,País_ESPANHA,País_ALEMANHA,País_ITALIA,País_MEXICO,Páginas,preço,nome,categoria,Autores
0,GALERA RECORD,8501112526,2018.0,1.0,1.0,0.0,BROCHURA,PORTUGUÊS,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,384.0,49.90,UM DE NÓS ESTÁ MENTINDO,TEEN,KAREN MCMANUS
1,GLOBO LIVROS,8525057517,2014.0,1.0,1.0,0.0,BROCHURA,PORTUGUÊS,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,384.0,59.90,1808,HISTÓRIA,LAURENTINO GOMES
2,GLOBAL EDITORA,8526008692,2006.0,1.0,1.0,0.0,BROCHURA,PORTUGUÊS,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,728.0,130.00,CASA-GRANDE & SENZALA,HISTÓRIA,GILBERTO FREYRE
3,PLANETA DO BRASIL,6555352795,2021.0,1.0,1.0,0.0,BROCHURA,PORTUGUÊS,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,288.0,38.42,SOMBRA E OSSOS,INFANTIL,LEIGH BARDUGO
4,USBORNE,140959386X,2015.0,1.0,1.0,0.0,CAPA DURA,PORTUGUÊS,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,20.0,49.90,MINHAS PRIMEIRAS PALAVRAS,INFANTIL,FELICITY BROOKS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3912,ALTA BOOKS,8576082675,2009.0,1.0,1.0,0.0,BROCHURA,PORTUGUÊS,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,440.0,114.00,CÓDIGO LIMPO,INFORMÁTICA E TECNOLOGIA,ROBERT C. MARTIN
3913,ALTA BOOKS,8576088487,2014.0,1.0,1.0,0.0,BROCHURA,PORTUGUÊS,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,408.0,98.00,DESENVOLVIMENTO DE APLICATIVOS ANDROID PARA LE...,INFORMÁTICA E TECNOLOGIA,DONN FELKER; MICHAEL BURTON
3914,ALTA BOOKS,855081587X,2021.0,1.0,1.0,0.0,BROCHURA,PORTUGUÊS,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,246.0,49.90,PYTHON PARA DATA SCIENCE PARA LEIGOS,INFORMÁTICA E TECNOLOGIA,JOHN PAUL MUELLER; LUCA MASSARON
3915,MYTHOS EDITORA,8578672518,2017.0,1.0,1.0,0.0,CAPA DURA,PORTUGUÊS,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,252.0,84.90,ZENITH - VOLUME 02,HQS,GRANT MORRISON


### Codificando Idioma

In [12]:
encoder=ce.OneHotEncoder(cols='Idioma',handle_unknown='return_nan',return_df=True,use_cat_names=True)
data_encoded = encoder.fit_transform(data_encoded)
data_encoded

/home/caio/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,Editora,ISBN,Ano,Edição,Origem_NACIONAL,Origem_IMPORTADO,Encadernação,Idioma_PORTUGUÊS,Idioma_INGLÊS,Idioma_PORT. PORTUGAL,...,País_PORTUGAL,País_ESPANHA,País_ALEMANHA,País_ITALIA,País_MEXICO,Páginas,preço,nome,categoria,Autores
0,GALERA RECORD,8501112526,2018.0,1.0,1.0,0.0,BROCHURA,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,384.0,49.90,UM DE NÓS ESTÁ MENTINDO,TEEN,KAREN MCMANUS
1,GLOBO LIVROS,8525057517,2014.0,1.0,1.0,0.0,BROCHURA,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,384.0,59.90,1808,HISTÓRIA,LAURENTINO GOMES
2,GLOBAL EDITORA,8526008692,2006.0,1.0,1.0,0.0,BROCHURA,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,728.0,130.00,CASA-GRANDE & SENZALA,HISTÓRIA,GILBERTO FREYRE
3,PLANETA DO BRASIL,6555352795,2021.0,1.0,1.0,0.0,BROCHURA,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,288.0,38.42,SOMBRA E OSSOS,INFANTIL,LEIGH BARDUGO
4,USBORNE,140959386X,2015.0,1.0,1.0,0.0,CAPA DURA,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,20.0,49.90,MINHAS PRIMEIRAS PALAVRAS,INFANTIL,FELICITY BROOKS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3912,ALTA BOOKS,8576082675,2009.0,1.0,1.0,0.0,BROCHURA,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,440.0,114.00,CÓDIGO LIMPO,INFORMÁTICA E TECNOLOGIA,ROBERT C. MARTIN
3913,ALTA BOOKS,8576088487,2014.0,1.0,1.0,0.0,BROCHURA,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,408.0,98.00,DESENVOLVIMENTO DE APLICATIVOS ANDROID PARA LE...,INFORMÁTICA E TECNOLOGIA,DONN FELKER; MICHAEL BURTON
3914,ALTA BOOKS,855081587X,2021.0,1.0,1.0,0.0,BROCHURA,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,246.0,49.90,PYTHON PARA DATA SCIENCE PARA LEIGOS,INFORMÁTICA E TECNOLOGIA,JOHN PAUL MUELLER; LUCA MASSARON
3915,MYTHOS EDITORA,8578672518,2017.0,1.0,1.0,0.0,CAPA DURA,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,252.0,84.90,ZENITH - VOLUME 02,HQS,GRANT MORRISON


### Codificando encadernação

In [13]:
encoder=ce.OneHotEncoder(cols='Encadernação',handle_unknown='return_nan',return_df=True,use_cat_names=True)
data_encoded = encoder.fit_transform(data_encoded)
data_encoded

/home/caio/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,Editora,ISBN,Ano,Edição,Origem_NACIONAL,Origem_IMPORTADO,Encadernação_BROCHURA,Encadernação_CAPA DURA,Encadernação_LÂMINA,Encadernação_FLEXIVEL,...,País_PORTUGAL,País_ESPANHA,País_ALEMANHA,País_ITALIA,País_MEXICO,Páginas,preço,nome,categoria,Autores
0,GALERA RECORD,8501112526,2018.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,384.0,49.90,UM DE NÓS ESTÁ MENTINDO,TEEN,KAREN MCMANUS
1,GLOBO LIVROS,8525057517,2014.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,384.0,59.90,1808,HISTÓRIA,LAURENTINO GOMES
2,GLOBAL EDITORA,8526008692,2006.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,728.0,130.00,CASA-GRANDE & SENZALA,HISTÓRIA,GILBERTO FREYRE
3,PLANETA DO BRASIL,6555352795,2021.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,288.0,38.42,SOMBRA E OSSOS,INFANTIL,LEIGH BARDUGO
4,USBORNE,140959386X,2015.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,20.0,49.90,MINHAS PRIMEIRAS PALAVRAS,INFANTIL,FELICITY BROOKS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3912,ALTA BOOKS,8576082675,2009.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,440.0,114.00,CÓDIGO LIMPO,INFORMÁTICA E TECNOLOGIA,ROBERT C. MARTIN
3913,ALTA BOOKS,8576088487,2014.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,408.0,98.00,DESENVOLVIMENTO DE APLICATIVOS ANDROID PARA LE...,INFORMÁTICA E TECNOLOGIA,DONN FELKER; MICHAEL BURTON
3914,ALTA BOOKS,855081587X,2021.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,246.0,49.90,PYTHON PARA DATA SCIENCE PARA LEIGOS,INFORMÁTICA E TECNOLOGIA,JOHN PAUL MUELLER; LUCA MASSARON
3915,MYTHOS EDITORA,8578672518,2017.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,252.0,84.90,ZENITH - VOLUME 02,HQS,GRANT MORRISON


In [14]:
data_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3917 entries, 0 to 3916
Data columns (total 34 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Editora                            3917 non-null   object 
 1   ISBN                               3917 non-null   object 
 2   Ano                                3917 non-null   float64
 3   Edição                             3917 non-null   float64
 4   Origem_NACIONAL                    3917 non-null   float64
 5   Origem_IMPORTADO                   3917 non-null   float64
 6   Encadernação_BROCHURA              3917 non-null   float64
 7   Encadernação_CAPA DURA             3917 non-null   float64
 8   Encadernação_LÂMINA                3917 non-null   float64
 9   Encadernação_FLEXIVEL              3917 non-null   float64
 10  Encadernação_CAPA DURA C/SOBRECAP  3917 non-null   float64
 11  Encadernação_ESPIRAL               3917 non-null   float

In [15]:
data_encoded['categoria'].nunique()

41

In [16]:
encoder= ce.BaseNEncoder(cols=['categoria'],return_df=True,base=5)
data_encoded=encoder.fit_transform(data_encoded)
data_encoded

/home/caio/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,Editora,ISBN,Ano,Edição,Origem_NACIONAL,Origem_IMPORTADO,Encadernação_BROCHURA,Encadernação_CAPA DURA,Encadernação_LÂMINA,Encadernação_FLEXIVEL,...,País_ITALIA,País_MEXICO,Páginas,preço,nome,categoria_0,categoria_1,categoria_2,categoria_3,Autores
0,GALERA RECORD,8501112526,2018.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,384.0,49.90,UM DE NÓS ESTÁ MENTINDO,0,0,0,1,KAREN MCMANUS
1,GLOBO LIVROS,8525057517,2014.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,384.0,59.90,1808,0,0,0,2,LAURENTINO GOMES
2,GLOBAL EDITORA,8526008692,2006.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,728.0,130.00,CASA-GRANDE & SENZALA,0,0,0,2,GILBERTO FREYRE
3,PLANETA DO BRASIL,6555352795,2021.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,288.0,38.42,SOMBRA E OSSOS,0,0,0,3,LEIGH BARDUGO
4,USBORNE,140959386X,2015.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,20.0,49.90,MINHAS PRIMEIRAS PALAVRAS,0,0,0,3,FELICITY BROOKS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3912,ALTA BOOKS,8576082675,2009.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,440.0,114.00,CÓDIGO LIMPO,0,1,2,3,ROBERT C. MARTIN
3913,ALTA BOOKS,8576088487,2014.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,408.0,98.00,DESENVOLVIMENTO DE APLICATIVOS ANDROID PARA LE...,0,1,2,3,DONN FELKER; MICHAEL BURTON
3914,ALTA BOOKS,855081587X,2021.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,246.0,49.90,PYTHON PARA DATA SCIENCE PARA LEIGOS,0,1,2,3,JOHN PAUL MUELLER; LUCA MASSARON
3915,MYTHOS EDITORA,8578672518,2017.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,252.0,84.90,ZENITH - VOLUME 02,0,1,2,4,GRANT MORRISON


### Codificando editora

In [17]:
data_encoded['Editora'].nunique()

502

In [18]:
encoder= ce.BaseNEncoder(cols=['Editora'],return_df=True,base=5)
data_encoded=encoder.fit_transform(data_encoded)
data_encoded

/home/caio/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,Editora_0,Editora_1,Editora_2,Editora_3,Editora_4,ISBN,Ano,Edição,Origem_NACIONAL,Origem_IMPORTADO,...,País_ITALIA,País_MEXICO,Páginas,preço,nome,categoria_0,categoria_1,categoria_2,categoria_3,Autores
0,0,0,0,0,1,8501112526,2018.0,1.0,1.0,0.0,...,0.0,0.0,384.0,49.90,UM DE NÓS ESTÁ MENTINDO,0,0,0,1,KAREN MCMANUS
1,0,0,0,0,2,8525057517,2014.0,1.0,1.0,0.0,...,0.0,0.0,384.0,59.90,1808,0,0,0,2,LAURENTINO GOMES
2,0,0,0,0,3,8526008692,2006.0,1.0,1.0,0.0,...,0.0,0.0,728.0,130.00,CASA-GRANDE & SENZALA,0,0,0,2,GILBERTO FREYRE
3,0,0,0,0,4,6555352795,2021.0,1.0,1.0,0.0,...,0.0,0.0,288.0,38.42,SOMBRA E OSSOS,0,0,0,3,LEIGH BARDUGO
4,0,0,0,1,0,140959386X,2015.0,1.0,1.0,0.0,...,0.0,0.0,20.0,49.90,MINHAS PRIMEIRAS PALAVRAS,0,0,0,3,FELICITY BROOKS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3912,0,0,0,1,4,8576082675,2009.0,1.0,1.0,0.0,...,0.0,0.0,440.0,114.00,CÓDIGO LIMPO,0,1,2,3,ROBERT C. MARTIN
3913,0,0,0,1,4,8576088487,2014.0,1.0,1.0,0.0,...,0.0,0.0,408.0,98.00,DESENVOLVIMENTO DE APLICATIVOS ANDROID PARA LE...,0,1,2,3,DONN FELKER; MICHAEL BURTON
3914,0,0,0,1,4,855081587X,2021.0,1.0,1.0,0.0,...,0.0,0.0,246.0,49.90,PYTHON PARA DATA SCIENCE PARA LEIGOS,0,1,2,3,JOHN PAUL MUELLER; LUCA MASSARON
3915,0,3,4,3,3,8578672518,2017.0,1.0,1.0,0.0,...,0.0,0.0,252.0,84.90,ZENITH - VOLUME 02,0,1,2,4,GRANT MORRISON


### Codificando autores

In [19]:
data_encoded['Autores'].nunique()

3008

In [20]:
encoder= ce.BaseNEncoder(cols=['Autores'],return_df=True,base=5)
data_encoded=encoder.fit_transform(data_encoded)
data_encoded

/home/caio/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,Editora_0,Editora_1,Editora_2,Editora_3,Editora_4,ISBN,Ano,Edição,Origem_NACIONAL,Origem_IMPORTADO,...,categoria_0,categoria_1,categoria_2,categoria_3,Autores_0,Autores_1,Autores_2,Autores_3,Autores_4,Autores_5
0,0,0,0,0,1,8501112526,2018.0,1.0,1.0,0.0,...,0,0,0,1,0,0,0,0,0,1
1,0,0,0,0,2,8525057517,2014.0,1.0,1.0,0.0,...,0,0,0,2,0,0,0,0,0,2
2,0,0,0,0,3,8526008692,2006.0,1.0,1.0,0.0,...,0,0,0,2,0,0,0,0,0,3
3,0,0,0,0,4,6555352795,2021.0,1.0,1.0,0.0,...,0,0,0,3,0,0,0,0,0,4
4,0,0,0,1,0,140959386X,2015.0,1.0,1.0,0.0,...,0,0,0,3,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3912,0,0,0,1,4,8576082675,2009.0,1.0,1.0,0.0,...,0,1,2,3,0,4,2,3,1,0
3913,0,0,0,1,4,8576088487,2014.0,1.0,1.0,0.0,...,0,1,2,3,0,4,4,0,1,3
3914,0,0,0,1,4,855081587X,2021.0,1.0,1.0,0.0,...,0,1,2,3,0,4,2,3,0,4
3915,0,3,4,3,3,8578672518,2017.0,1.0,1.0,0.0,...,0,1,2,4,0,4,3,2,0,0


### Removendo campos desnecessários

In [21]:
data_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3917 entries, 0 to 3916
Data columns (total 46 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Editora_0                          3917 non-null   int64  
 1   Editora_1                          3917 non-null   int64  
 2   Editora_2                          3917 non-null   int64  
 3   Editora_3                          3917 non-null   int64  
 4   Editora_4                          3917 non-null   int64  
 5   ISBN                               3917 non-null   object 
 6   Ano                                3917 non-null   float64
 7   Edição                             3917 non-null   float64
 8   Origem_NACIONAL                    3917 non-null   float64
 9   Origem_IMPORTADO                   3917 non-null   float64
 10  Encadernação_BROCHURA              3917 non-null   float64
 11  Encadernação_CAPA DURA             3917 non-null   float

In [22]:
data_encoded = data_encoded.drop(columns=['nome','ISBN'],axis=1)
data_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3917 entries, 0 to 3916
Data columns (total 44 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Editora_0                          3917 non-null   int64  
 1   Editora_1                          3917 non-null   int64  
 2   Editora_2                          3917 non-null   int64  
 3   Editora_3                          3917 non-null   int64  
 4   Editora_4                          3917 non-null   int64  
 5   Ano                                3917 non-null   float64
 6   Edição                             3917 non-null   float64
 7   Origem_NACIONAL                    3917 non-null   float64
 8   Origem_IMPORTADO                   3917 non-null   float64
 9   Encadernação_BROCHURA              3917 non-null   float64
 10  Encadernação_CAPA DURA             3917 non-null   float64
 11  Encadernação_LÂMINA                3917 non-null   float

### Testando novamente o classificador

In [23]:
y = data_encoded['preço']
X = data_encoded.drop(columns=['preço'],axis=1)

In [24]:
X

,Editora_0,Editora_1,Editora_2,Editora_3,Editora_4,Ano,Edição,Origem_NACIONAL,Origem_IMPORTADO,Encadernação_BROCHURA,...,categoria_0,categoria_1,categoria_2,categoria_3,Autores_0,Autores_1,Autores_2,Autores_3,Autores_4,Autores_5
0,0,0,0,0,1,2018.0,1.0,1.0,0.0,1.0,...,0,0,0,1,0,0,0,0,0,1
1,0,0,0,0,2,2014.0,1.0,1.0,0.0,1.0,...,0,0,0,2,0,0,0,0,0,2
2,0,0,0,0,3,2006.0,1.0,1.0,0.0,1.0,...,0,0,0,2,0,0,0,0,0,3
3,0,0,0,0,4,2021.0,1.0,1.0,0.0,1.0,...,0,0,0,3,0,0,0,0,0,4
4,0,0,0,1,0,2015.0,1.0,1.0,0.0,0.0,...,0,0,0,3,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3912,0,0,0,1,4,2009.0,1.0,1.0,0.0,1.0,...,0,1,2,3,0,4,2,3,1,0
3913,0,0,0,1,4,2014.0,1.0,1.0,0.0,1.0,...,0,1,2,3,0,4,4,0,1,3
3914,0,0,0,1,4,2021.0,1.0,1.0,0.0,1.0,...,0,1,2,3,0,4,2,3,0,4
3915,0,3,4,3,3,2017.0,1.0,1.0,0.0,0.0,...,0,1,2,4,0,4,3,2,0,0


In [25]:
scaler = MinMaxScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [26]:
model = LinearRegression()
scores = cross_val_score(model, X_scaled, y, cv=5,scoring='r2')
print(scores)
print(f'Pontuação r2 média com normalização: {scores.mean()}')

[0.31983876 0.16466174 0.45213296 0.0377561  0.29449167]
Pontuação r2 média com normalização: 0.2537762463161036


In [ ]:
poly = PolynomialFeatures(4)
X_scaled_poly = poly.fit_transform(X_scaled)
model = LinearRegression()
scores = cross_val_score(model, X_scaled_poly, y, cv=5,scoring='r2')
print(scores)
print(f'Pontuação r2 média com normalização: {scores.mean()}')

Error: Session cannot generate requests